# Import The Required Libraries

In [1]:
import json
import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load JSON File

In [2]:
JSON_PATH = "result.json"

In [4]:
json_path = Path(JSON_PATH)

raw_text = json_path.read_text(encoding="utf-8")

try:
    # Standard JSON (list or dict)
    data = json.loads(raw_text)
    if isinstance(data, dict) and "messages" in data:
        messages = data["messages"]
    else:
        messages = data
    print(f"Loaded {len(messages)} messages from JSON file.")
except json.JSONDecodeError:
    # Fallback: JSONL
    messages = []
    for i, line in enumerate(raw_text.splitlines(), start=1):
        line = line.strip()
        if not line:
            continue
        try:
            messages.append(json.loads(line))
        except json.JSONDecodeError:
            print(f"Skipping invalid JSON at line {i}")
    print(f"Loaded {len(messages)} messages from JSONL file.")

Loaded 4596 messages from JSON file.


# Flatten Telegram Text

In [12]:
rows = []

for msg in messages:
    # Combine Telegram text or text_entities into a single string
    if isinstance(msg.get("text"), list):
        full_text = "".join(
            t if isinstance(t, str) else t.get("text", "")
            for t in msg["text"]
        )
    elif isinstance(msg.get("text"), str):
        full_text = msg["text"]
    else:
        full_text = ""

    rows.append({
        "message_id": msg.get("id"),
        "date": msg.get("date"),
        "full_text": full_text
    })

records = pd.DataFrame(rows)

# Convert date to datetime (safe conversion)
records["date"] = pd.to_datetime(records["date"], errors="coerce")

records.head()

,message_id,date,full_text
0,1,2021-09-03 13:57:38,
1,12,2021-09-05 00:24:19,#خبر_مهم\n\nسلام به همه‌ی علموصی ها😍👊🏾\n\n✅این...
2,13,2021-09-05 00:24:25,
3,14,2021-09-05 00:34:39,🧑‍🏫 سید محمد شهرتاش\n🏫 #برق\n📒 بررسی ۱-بررسی۲-...
4,15,2021-09-05 00:35:51,🧑‍🏫 دکتر هاجر فلاحتی\n🏫 #مهندسی_کامپیوتر\n📒 مد...
